In [1]:
!rm -rf Eunomia
!git clone https://github.com/ouaguenouni/Eunomia
!pip install -r Eunomia/requirements.txt

Cloning into 'Eunomia'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (532/532), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 532 (delta 396), reused 458 (delta 322), pack-reused 0
Receiving objects: 100% (532/532), 3.49 MiB | 9.87 MiB/s, done.
Resolving deltas: 100% (396/396), done.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Eunomia.preferences import *
from Eunomia.additive_functions import *
from Eunomia.alternatives import *
from Eunomia.sampling import *
from Eunomia.mcmc import *
from Eunomia.degree import *
from Eunomia.experiments import *


In [ ]:
import time

TEST_NAME = "SX_IJCAI_HYPERPARAMETER_NOISY_3"
REPLACE = False

n = 4
sigma_weights = 1
sigma_w_vals = np.linspace(1e-2, 2, 10)
sigma_p_vals =  np.linspace(1e-2, 2, 10)
n_samples = 500
k=2

for iter_ in range(10):
    for sigma_w in sigma_p_vals:
        for sigma_p in sigma_w_vals:
            for m in [2**(n-1)]:
                run_d = {
                    "iter":iter_,
                    "n": n,
                    "k": k,
                    "m": m,
                    "sigma_w": float(sigma_w),
                    "sigma_p": float(sigma_p),
                    "n_samples": n_samples
                }
                found = find_experiment_file(run_d, TEST_NAME)
                if found:
                    date = found
                    print(f"Found file: {file_name}")
                    if REPLACE:
                        print('Replacing it...')
                    else:
                        continue
                else:
                    file_name = compute_experiment_file_name(run_d, TEST_NAME)
                    print("Registering the results in: ", file_name)
                ####
                theta = generate_additive_theta(n, k)
                weights = generate_normal_weights(theta, sigma_weights)
                alternatives = generate_random_alternatives_matrix(m, n)
                ranks = compute_ws_ranks(alternatives, theta, weights) + np.random.normal(0, 1)
                t_sv = compute_semivalues(n, theta, weights, lambda x:1)
                preferences = PreferenceModel(alternatives, ranks)
                data = preferences.generate_preference_matrix(theta)
                data = torch.tensor(data).float()
                t = time.time()
                model = posterior_sampling_model(data, sigma_w=sigma_w, sigma_p=sigma_p)
                diag, sampled_weights, sigmas = sample_model(model, data, "w", "sigma",n_chains = 5, warmup_steps=500, num_samples=n_samples, return_diag=True)
                t = time.time() - t
                accs_d = get_acc_distribution(data, sampled_weights, sigmas)
                predicted_rankings = [np.argsort(compute_semivalues(n, theta, weights, lambda x:1))[::-1] for weights in sampled_weights]
                kt_d = get_kt_distribution(predicted_rankings, np.argsort(t_sv))
                ###
                
                file_path = record_experiment_results(run_d, TEST_NAME)
                
                run_d["time"] = t
                run_d["weights"] = weights.tolist()
                run_d["predicted_rankings"] = [i.tolist() for i in predicted_rankings]
                run_d["accuracy_distribution"] = [i.tolist() for i in accs_d]
                run_d["kt_d"] = [i.tolist() for i in kt_d]
                run_d["acceptance_rate"] = diag["acceptance rate"]["chain 0"]
                run_d["w_eff"] = [float(i) for i in diag["w"]["n_eff"].numpy()]
                run_d["w_rhat"] = [float(i) for i in diag["w"]["r_hat"].numpy()]
                
                print("diag: ", diag)
                with open(file_path, 'w') as file:
                    yaml.dump(run_d, file, default_flow_style=False)
    

Registering the results in:  SX_IJCAI_HYPERPARAMETER_NOISY_3/2024-01-11/iter-0_n-4_k-2_m-8_sigma_w-0.01_sigma_p-0.01_n_samples-500.yaml


Sample: 100%|████████████████████████████| 1000/1000 [00:27, 36.96it/s, step size=5.10e-01, acc. prob=0.909]


diag:  {'sigma': OrderedDict([('n_eff', tensor(682.9730)), ('r_hat', tensor(0.9983))]), 'w': OrderedDict([('n_eff', tensor([ 910.5498, 1039.2109, 1440.6317, 1407.9006, 1163.0095,  939.8123,
         944.0267,  989.1652, 1221.2603, 1036.0496])), ('r_hat', tensor([0.9983, 0.9983, 0.9980, 0.9980, 0.9982, 1.0002, 0.9991, 0.9986, 1.0039,
        0.9980]))]), 'divergences': {'chain 0': []}, 'acceptance rate': {'chain 0': 1.0}}
Registering the results in:  SX_IJCAI_HYPERPARAMETER_NOISY_3/2024-01-11/iter-0_n-4_k-2_m-8_sigma_w-0.01_sigma_p-0.23111111111111113_n_samples-500.yaml


Sample: 100%|████████████████████████████| 1000/1000 [01:06, 15.01it/s, step size=1.57e-01, acc. prob=0.784]


diag:  {'sigma': OrderedDict([('n_eff', tensor(95.3773)), ('r_hat', tensor(1.0092))]), 'w': OrderedDict([('n_eff', tensor([304.6518, 193.1476, 202.6935, 301.1165, 291.3003, 131.5648, 215.4104,
        348.2952, 221.9003, 127.0905])), ('r_hat', tensor([0.9982, 1.0011, 1.0052, 0.9984, 1.0033, 1.0124, 1.0106, 0.9987, 0.9983,
        1.0001]))]), 'divergences': {'chain 0': [375]}, 'acceptance rate': {'chain 0': 0.972}}
Registering the results in:  SX_IJCAI_HYPERPARAMETER_NOISY_3/2024-01-11/iter-0_n-4_k-2_m-8_sigma_w-0.01_sigma_p-0.45222222222222225_n_samples-500.yaml


...

Sample: 100%|████████████████████████████| 1000/1000 [00:47, 21.23it/s, step size=1.33e-01, acc. prob=0.650]


diag:  {'sigma': OrderedDict([('n_eff', tensor(45.3460)), ('r_hat', tensor(1.0210))]), 'w': OrderedDict([('n_eff', tensor([ 87.2830, 110.7953, 111.1179,  90.5679,  67.9982,  39.2549,  35.5061,
        175.7259,  86.7000,  55.1172])), ('r_hat', tensor([1.0083, 1.0132, 1.0007, 1.0077, 0.9980, 1.0449, 1.0043, 0.9995, 1.0085,
        1.0330]))]), 'divergences': {'chain 0': [16, 84, 177, 184, 279, 302, 362]}, 'acceptance rate': {'chain 0': 0.868}}
Registering the results in:  SX_IJCAI_HYPERPARAMETER_NOISY_3/2024-01-12/iter-3_n-4_k-2_m-8_sigma_w-1.1155555555555556_sigma_p-1.3366666666666667_n_samples-500.yaml


Sample: 100%|████████████████████████████| 1000/1000 [02:00,  8.27it/s, step size=8.11e-02, acc. prob=0.613]


diag:  {'sigma': OrderedDict([('n_eff', tensor(39.6596)), ('r_hat', tensor(1.0038))]), 'w': OrderedDict([('n_eff', tensor([ 71.8579,  65.9971,  22.1992,  89.7956,  62.4206,  28.1249, 233.9946,
         47.1741,  37.7646,  71.1400])), ('r_hat', tensor([1.0118, 0.9980, 1.0073, 1.0304, 1.0062, 1.0034, 0.9994, 1.0177, 1.0585,
        1.0058]))]), 'divergences': {'chain 0': [230, 288, 441, 443, 464]}, 'acceptance rate': {'chain 0': 0.806}}
Registering the results in:  SX_IJCAI_HYPERPARAMETER_NOISY_3/2024-01-12/iter-3_n-4_k-2_m-8_sigma_w-1.1155555555555556_sigma_p-1.557777777777778_n_samples-500.yaml


Warmup:  18%|█████▏                       | 181/1000 [00:17, 11.05it/s, step size=8.16e-02, acc. prob=0.781]